In [126]:
import pandas as pd
import numpy as np

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show
#from bokeh.sampledata.stocks import AAPL
days_per_year = 52 * 5

In [127]:
kaggle_file = "data/archive/ETFs/spy.us.txt"
other_file = "data/Download Data - INDEX_US_S&P US_SPX.csv"
hist=pd.read_csv(other_file)
print(hist)


           Date      Open      High       Low     Close
0    02/28/2023  3,977.19  3,997.50  3,968.98  3,970.15
1    02/27/2023  3,992.36  4,018.05  3,973.55  3,982.24
2    02/24/2023  3,973.24  3,978.25  3,943.08  3,970.04
3    02/23/2023  4,018.60  4,028.30  3,969.19  4,012.32
4    02/22/2023  4,001.83  4,017.37  3,976.90  3,991.05
..          ...       ...       ...       ...       ...
247  03/04/2022  4,342.12  4,342.12  4,284.98  4,328.87
248  03/03/2022  4,401.31  4,416.78  4,345.56  4,363.49
249  03/02/2022  4,322.56  4,401.48  4,322.56  4,386.54
250  03/01/2022  4,363.14  4,378.45  4,279.54  4,306.26
251  02/28/2022  4,354.17  4,388.84  4,315.12  4,373.94

[252 rows x 5 columns]


In [128]:
dates = np.array(hist['Date'], dtype=np.datetime64)
date_close_lut = dict(date=dates, close=hist['Close'])
source = ColumnDataSource(data=date_close_lut)

p = figure(height=700,
            width=1900, 
            tools="xpan", 
            toolbar_location=None,
            x_axis_type="datetime", 
            x_axis_location="above",
           background_fill_color="#efefef", 
           x_range=(dates[1500], dates[2500])
           )

p.line('date', 'close', source=source)
p.yaxis.axis_label = 'Price'

select = figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=200,
    width=1900,
    y_range=p.y_range,
    x_axis_type="datetime",
    y_axis_type=None,
    tools="",
    toolbar_location=None,
    background_fill_color="#efefef"
)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'close', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool



ValueError: Cannot create a NumPy datetime other than NaT with generic units

In [ ]:
def future_below_yr(years, df):
    # dataset_1 = "%Y-%m-%d"
    td = years * days_per_year
    return pd.DataFrame(
            [
                {
                    "date": df['date'][i], 
                    "close": df['close'][i]
                }
                for i in range(len(df['date'])) 
                if df['close'][i] < df['close'][max(0.0, i-td)] 
            ]
        )

In [ ]:
years_set_back = 10
p.circle(
    'date', 
    'close', 
    source=future_below_yr(years_set_back, date_close_lut), 
    size=20, 
    color="red", 
    alpha=0.1
)
print(type(hist))

In [ ]:
show(column(p, select))